In [2]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Load data
df = pd.read_csv("https://raw.githubusercontent.com/JoshData/historical-state-population-csv/refs/heads/primary/historical_state_population_by_year.csv", names = ["state","year","population"])

# Sort out unique years
unique_years = sorted(df['year'].unique())

# Helper functions
def create_choropleth_map(data, selected_year):
    dff = data[data['year'] == selected_year]
    fig = px.choropleth(
        dff,
        locations='state',
        locationmode="USA-states",
        color='population',
        scope="usa",
        color_continuous_scale="Viridis",
        hover_name='state',
        labels={'population': 'Population'},
        range_color=(dff['population'].min(), dff['population'].max())
    )
    fig.update_layout(
        margin=dict(l=0, r=0, t=30, b=0),
        geo=dict(bgcolor='rgba(0,0,0,0)')
    )
    return fig

def create_line_chart(data, selected_state):
    dff = data[data['state'] == selected_state].sort_values('year')
    fig = px.line(
        dff,
        x='year',
        y='population',
        title=f"Population Over Years: {selected_state}",
        labels={'year': 'Year', 'population': 'Population'}
    )
    fig.update_layout(margin=dict(l=0, r=0, t=40, b=0))
    return fig

# Initialize app
app = dash.Dash(__name__)
server = app.server

# Layout
app.layout = html.Div([
    html.H1("US State Population Dashboard", style={'textAlign': 'center'}),
    
    html.Div([
        html.Label("Select Year:"),
        dcc.Dropdown(
            id='year-dropdown',
            options=[{'label': str(year), 'value': year} for year in unique_years],
            value=unique_years[-1],
            clearable=False
        ),
    ], style={'width': '20%', 'display': 'inline-block', 'verticalAlign': 'top'}),
    
    dcc.Graph(
        id='choropleth-map',
        style={'width': '70%', 'display': 'inline-block', 'height': '600px'}
    ),
    
    dcc.Graph(
        id='line-chart',
        style={'width': '90%', 'margin': '0 auto', 'display': 'block', 'height': '400px'}
    ),
])

# Callbacks
@app.callback(
    Output('choropleth-map', 'figure'),
    Input('year-dropdown', 'value')
)
def update_choropleth(selected_year):
    return create_choropleth_map(df, selected_year)

@app.callback(
    Output('line-chart', 'figure'),
    Input('choropleth-map', 'clickData'),
    Input('year-dropdown', 'value')
)
def update_line_chart(clickData, selected_year):
    if clickData is not None:
        clicked_state = clickData['points'][0]['location']
        return create_line_chart(df, clicked_state)
    else:
        return px.line(title="Click a state on the map to see its population over the years")

if __name__ == '__main__':
    # For a normal script, run:
    app.run_server(debug=True, port=8050)
